In [2]:
import numpy as np

from math import sqrt

from data_utils import DataUtils

In [3]:
train_data_ins = DataUtils(data_source='./ag_news_csv/train.csv')
train_data_ins.load_data()
train_data_ins.shuffle_data()

Data loaded from ./ag_news_csv/train.csv


In [4]:
batch_indices, classes = train_data_ins.get_batch_to_indices()

batch_indices[0], len(batch_indices[0]), len(batch_indices), batch_indices.shape

(array([40, 13, 19, ...,  0,  0,  0]), 1014, 128, (128, 1014))

In [5]:
import tensorflow as tf

In [6]:
input_size=1014
alphabet_size=69
num_of_classes=4

In [7]:
input_x = tf.placeholder(tf.int64, shape=[None, input_size], name='input_x')
input_y = tf.placeholder(tf.float32, shape=[None, num_of_classes], name='input_y')

In [8]:
Q = tf.concat(
                [
                    # # Zero padding vector for out of alphabet characters
                    tf.zeros([1, alphabet_size]),
                    # one-hot vector representation for alphabets
                    tf.one_hot(list(range(alphabet_size)), alphabet_size, 1.0, 0.0)

                ],
                0,
                name='Q')
x = tf.nn.embedding_lookup(Q, input_x)
# Add the channel dim, thus the shape of x is [batch_size, input_size, alphabet_size, 1]
x = tf.expand_dims(x, -1)
filter_width = x.get_shape()[2].value

In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    res = sess.run(x, feed_dict={input_x: batch_indices})

In [10]:
len(res[0][0])

69

In [11]:
res.shape

(128, 1014, 69, 1)

In [12]:
len(res), len(res[0])

(128, 1014)

In [13]:
cl = [256, 7, 3]

In [14]:
filter_shape = [cl[1], filter_width, 1, cl[0]]
filter_shape

[7, 69, 1, 256]

In [15]:
stdv = 1 / sqrt(cl[0] * cl[1])

In [16]:
stdv

0.0236227795630767

In [17]:
sess = tf.Session()
input = tf.get_variable('input',shape=[128, 1014, 69, 1],initializer=tf.random_uniform_initializer())
filter= tf.get_variable('filter',shape=[7, 69, 1, 256],initializer=tf.truncated_normal_initializer())
op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')
sess.run(tf.global_variables_initializer())
input_res = sess.run(input)
filter_res = sess.run(filter)
op_res = sess.run(op)


In [18]:
filter_res.shape

(7, 69, 1, 256)

In [19]:
op_res.shape

(128, 1008, 1, 256)